# Zusatzkapitel 2 - Neuronale Netze mit PyTorchText

## Z.2.1. Kapitelübersicht <a class="anchor" id="Z-2-1"/>

In diesem Kapitel zeige ich, wie man einfache Neuronale Netze mit der Deep-Learning-Bibliothek **PyTorch** und **TorchText** erstellt. Dieses Kapitel baut auf den Kapiteln 10 bis 13 auf, d.h. das Konzept von Neuronalen Netzen, gängige Deep Learning Begriffe und erste Praxiserfahrungen sollten bekannt sein. Weiterhin sollte das Konzept der objektorientierten Programmierung in Python geläufig sein, da PyTorch eine Klassenarchitektur zur Erstellung von Modellen benutzt. Auch wird die Benutzung einer GPU wie bei Kapitel 12 dringend empfohlen (auch hier kann **Google Colab** wieder benutzt werden).

**Hinweis**: Das PyTorch-Modul befindet sich <u>nicht</u> in der bei dieser Tutorialreihe beigelegten `requirements.txt` Datei. Es muss eigenständig installiert werden, was jedoch im Gegensatz zu Keras oder Tensorflow weitaus unkomplizierter ist. Die Webseite von <a href="https://pytorch.org/">PyTorch</a> ist hier sehr hilfreich. <a href="https://github.com/pytorch/text">TorchText</a> muss ebenfalls noch installiert werden.

<b>Abschnittsübersicht</b>:<br>
[Z.2.1. Kapitelübersicht](#Z-2-1)<br>
[Z.2.2. Übersicht zu PyTorch](#Z-2-2)<br>
[Z.2.3. Implementierung in PyTorch](#Z-2-3)<br>
[Z.2.3.1. Aufteilung des Korpus](#Z-2-3-1)<br>
[Z.2.3.2. Laden der JSON-Dateien](#Z-2-3-2)<br>
[Z.2.3.3. Erstellung des Modells](#Z-2-3-3)<br>

Am Ende dieses Kapitel werden wir folgende Themen behandelt und/oder vertieft haben:
- Implementierung eines Neuronalen Netz mit PyTorch

## Z.2.2. Übersicht zu PyTorch <a class="anchor" id="Z-2-2"/>

Wie auch **Keras** ist **PyTorch** eine Deep-Learning-Bibliothek. Anders als Keras basiert PyTorch nicht auf der Bibliothek **Tensorflow**, sondern auf der Bibliothek **Torch**. PyTorch unterscheidet sich zu Keras, es erfordert eine stärkere Auseinandersetzung mit den Mechaniken von Deep Learning, weshalb auch mehr Code als bei Keras benötigt wird, in welchem meist mit nur wenigen Zeilen Code ein Neuronales Netz erstellt werden konnte. Im akademischen Bereich wird PyTorch aktuell (Stand März 2020) jedoch viel häufiger als Keras verwendet, da es eine bessere Performance und die Erstellung von komplexeren Architekturen ermöglicht. Keras selbst wird anders als Tensorflow oder PyTorch aufgrund seiner Simplizität meistens nur zur Erstellung von Prototypen genutzt, abseits von Tutorials ist es deshalb seltener zu finden. Sollte man mit Textdaten arbeiten, empfiehlt es sich, neben PyTorch noch die ebenfalls von PyTorch bereitgestelle Bibliothek **TorchText** zu verwenden. Diese vereinfacht typische Vorverarbeitungsschritte für die Arbeit mit Textdaten ungemein. 


TODO

## Z.2.3. Implementierung in PyTorch <a class="anchor" id="Z-2-3"/>

In [99]:
from nltk import word_tokenize
from nltk.corpus import stopwords
punctuation = ['!', '#','$','%','&', "'", '(',')','*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', 
               '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '`', '``', 'wurde', 'wurden']
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils import data
from torchtext import data, datasets

### Z.2.3.1. Aufteilung des Korpus <a class="anchor" id="Z-2-3-1"/>

Das Einlesen des Korpus und die Aufteilung des Korpus unterscheidet sich von dem Vorgehen in den anderen Kapiteln. Viele Klassen und Methoden von PyTorch und TorchText erwarten verschiedene Dateien für den Trainings-, Test- und Validierungsdatensatz. Mithilfe der uns bereits bekannten `train_test_split`-Funktion von Scikit-learn können wir diese Dateien erstellen, da die Funktion als Eingabe neben numpy-Matrizen auch DataFrames erlaubt. Bis jetzt hatten wir den Datei-Typen **csv** verwendet. Dieser ist jedoch für die Nutzung mit TorchText problematisch, die JSON Variante <a href="http://jsonlines.org/">JSON Lines</a> eignet sich hier besser.[<sup>1</sup>](#fn1) Beim **JSON Line** Format ist jede Zeile ein valider JSON Wert. Für dieses Tutorial wurden die Spalten "id" und "length" aus dem Korpus entfernt.


<hr style="border: 0.1px solid black;"/>
<span id="fn1" style="font-size:8pt; line-height:1"><sup style="font-size:5pt">1</sup> &nbsp; 
Unter anderem liegt das daran, dass csv-Dateien mit Kommas voneinander getrennt werden und TorchText Probleme hat, diese Trennungskommas von Kommas in der "Text"-Spalte zu unterschieden. Für weitere Erklärungen zum Vorteil von JSON-Dateien gegenüber csv-Dateien siehe dieses <a href="https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/A%20-%20Using%20TorchText%20with%20Your%20Own%20Datasets.ipynb">TorchText-Tutorial</a>.

In [87]:
corpus = pd.read_csv("tutorialdata/corpora/wikicorpus_v2.csv", index_col = 0)
corpus = corpus.drop("id", axis=1)
corpus = corpus.drop("length", axis=1)

In [88]:
corpus.head()

,category,text
0,Album nach Typ,All the Best ! ( englisch Alles Gute ! ) ist d...
1,Album nach Typ,Let It Roll : Songs by George Harrison ist das...
2,Album nach Typ,Lieder wie Orkane ist das dritte offizielle Be...
3,Album nach Typ,Long Stick Goes Boom : The Anthology ist eine ...
4,Album nach Typ,Los Grandes Éxitos en Español ( spanisch für D...


In [89]:
X_train, X_remain = train_test_split(corpus, 
                                     test_size=0.4, 
                                     train_size=0.6,
                                     random_state=42,
                                     stratify=corpus["category"])
X_val = X_remain[:1200]
X_test = X_remain[1200:]


def df_to_jsonl(df, filename, column="text", path="tutorialdata/pytorch_data/"):
    """ DataFrame with text column to Json Line Format. """

    df[column] = df.apply(lambda row: word_tokenize(row[column]), axis=1)
    df.to_json(f"{path}{filename}.json", orient='records', lines=True)
    
    
df_to_jsonl(X_train, "train")
df_to_jsonl(X_val, "val")
df_to_jsonl(X_test, "test")

<ipython-input-89-c898e509b3c3>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df.apply(lambda row: word_tokenize(row[column]), axis=1)


### Z.2.3.2. Laden der JSON-Dateien <a class="anchor" id="Z-2-3-2"/>

Eines der Hauptkonzepte von TorchText ist das sogenannte **`Field`**. Dadurch wird definiert, wie Daten verarbeitet werden sollen. So kann z.B. angegeben werden, ob Texte in diesem `Field` tokenisiert werden sollen und nur kleingeschrieben verarbeitet werden sollen. Im `Field` Texte wurde als Tokenizer *toktok* angegeben, eine Tokenizer der NLP-Bibliothek **NLTK** (andere mögliche Werte finden sich <a href="https://pytorch.org/text/_modules/torchtext/data/utils.html">hier</a>). In unserem Fall ändert sich durch die Angabe des Tokenizers nichts an den Texten der Datensatz-Dateien, da diese bereits vorher wegen der Konvertierung ins JSON Lines Format tokenisiert wurden.[<sup>2</sup>](#fn2) Weiterhin haben wir alle Wörter mit dem Argument `lower` in Kleinbuchstaben konvertiert.[<sup>3</sup>](#fn3) Auch Stoppwörter sowie viele Satzzeichen wurden ignoriert.<br>
Neben der Übergabe von Argumenten gibt es von `Field` noch verschiedene Varianten. Für die Spalte mit den Klassen verwenden wir beispielsweise die Variante `LabelField`. Im Grunde unterscheidet diese sich nicht groß von `Field`, jedoch sind die Argumente `sequential` und `unk_token` auf `None` gesetzt. Dies macht Sinn, da es sich bei unseren Labeln nicht um sequentielle Daten (*hier*: keine ganzen Texte) handelt und es auch nicht zu einem Out-of-Vocabulary (OOV) Fehler kommen kann. Anders ist dies jedoch bei unserem TEXT-`Field`. Weiter unten geben wir die maximale Größe unseres Vokabulars an, d.h. wir begrenzen die von unserem Modell zu verarbeitenden Wörter auf die 25.000 häufigsten Wörter. Geben wir jedoch die tatsächliche Größe unseres Vokabulars aus, nachdem wir es mit `build_vocab` erstellt haben, erhalten wir den Wert 25.002. Dies liegt daran, dass noch die zwei zusätzlichen Vokabeln `<unk>` und `<pad>` hinzugefügt. Da wir nur das Vokabular unseres Trainingsdatensatzes kennen, kann es sein, dass es im Validierungs- oder Testdatensatz Wörter gibt, die nicht in unserem Vokabular vorkommen. Um nicht den OOV-Fehler zu erhalten, wird von PyTorch die Vokabel `<unk>` verwendet (= unknown). Der `<pad>`-Token (= padding) wird zum "Auffüllen" von Batches verwendet.[<sup>4</sup>](#fn4) 

<br>

Mit der Funktion `data.TabularDataset.splits` rufen wir die verschiedenen JSON-Dateien als Tabellen-Datensatz auf. Dem Argument `fields` übergeben wir ein Dictionary `assigned_fields`, in welchen wir unseren erstellten `Fields` TEXT und CATEGORIES die entsprechenden zugehörigen Spalten unserer Datensätze zuordnen. In der späteren Trainingsschleife können wir uns das zu Nutze machen und über z.B. `batch.category` auf die Kategorien zugreifen. Mit dem `skip_header = True` geben wir an, dass die Spaltenbezeichnungen ignoriert werden.


<hr style="border: 0.1px solid black;"/>
<span id="fn2" style="font-size:8pt; line-height:1"><sup style="font-size:5pt">2</sup> &nbsp; Texte sollten im besten Fall vorher tokenisiert werden, da ansonsten TorchText jedes Mal, wenn die Datensätze geladen werden, diese neu tokenisiert. Dies kann bei großen Datensätzen sehr viel Zeit kosten.<br>
<span id="fn3" style="font-size:8pt; line-height:1"><sup style="font-size:5pt">3</sup> &nbsp; Dies ist für die deutsche Sprache nicht immer sinnvoll, wurde hier jedoch verwendet, um eine diverseres Vokabular zu erhalten.<br>
<span id="fn4" style="font-size:8pt; line-height:1"><sup style="font-size:5pt">4</sup> &nbsp; Die GPU (oder CPU) verarbeitet die Trainingsdaten beim Training des Neuronalen Netzes in <b>batches</b> (= Stapel), die alle eine gewisse Länge haben. Ist ein Batch zu klein, wird er mit Padding-Tokens aufgefüllt.

In [90]:
stop_words = stopwords.words('german') + punctuation 

TEXT = data.Field(tokenize = "toktok",
                  lower = True,
                  stop_words=stop_words)
CATEGORIES = data.LabelField()
assigned_fields = {"text": ('text', TEXT), 
                   "category": ('category', CATEGORIES)}

train, val, test = data.TabularDataset.splits(path='tutorialdata/pytorch_data/', 
                                              train='train.json',
                                              validation='val.json', 
                                              test='test.json', 
                                              format='json',
                                              fields=assigned_fields,
                                              skip_header = True)

In [91]:
example = vars(train.examples[1])
print(example["text"][:20])

['satz', 'trachtenbrot', 'benannt', 'boris', 'trachtenbrot', 'satz', 'mathematischen', 'logik', '1950', 'bewiesen', 'besagt', 'endlichen', 'modellen', 'allgemeingültigen', 'sätze', 'prädikatenlogik', 'erster', 'stufe', 'rekursiv', 'aufzählbar']


#### Begrenzung des Vokabulars

In [92]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train, max_size = MAX_VOCAB_SIZE)
CATEGORIES.build_vocab(train)

In [93]:
print(f"Einzigarte Tokens im TEXT-Vokabular: {len(TEXT.vocab)}")
print(f"Einzigarte Tokens im CATEGORIES-Vokabular: {len(CATEGORIES.vocab)}")

Einzigarte Tokens im TEXT-Vokabular: 25002
Einzigarte Tokens im CATEGORIES-Vokabular: 30


In [94]:
print(TEXT.vocab.freqs.most_common(20))

[('jedoch', 2720), ('sowie', 2706), ('jahr', 2678), ('the', 2566), ('2', 2385), ('seit', 2320), ('zwei', 2272), ('1', 2258), ('stadt', 2243), ('ab', 2203), ('jahre', 2103), ('mehr', 2056), ('etwa', 1994), ('zeit', 1963), ('of', 1826), ('ersten', 1820), ('dabei', 1712), ('jahren', 1677), ('drei', 1671), ('a', 1659)]


Der letzte Schritt bei der Vorbereitung der Daten ist die Erstellung der **Iteratoren**. Ein Datensatz-Iterator ermöglicht das einfache Laden von Daten in Neuronalen Netzwerke und hilft bei der Organisation von Stapelverarbeitung (= batching), Konvertierung und Maskierung. Wir iterieren über diese Iteratoren in der Trainings-/Evaluierungsschleife und sie geben bei jeder Iteration einen Batch von Beispielen zurück (indiziert und in Tensoren umgewandelt). Dafür verwenden wir den sogenannten `BucketIterator`, der einen Batch von Beispielen zurückgibt, bei der jedes Beispiel eine ähnliche Länge hat. Somit wird der Gebrauch von Padding (*hier*: den `<pad>` Tojens) minimiert. Sollte eine GPU vorhanden sein und CUDA ist erfolgreich installiert worden, kann mithilfe von `torch.device` diese genutzt werden, indem sie `device` dem `BucketIterator` übergeben wird. Zuletzt geben wir ähnlich wie bei Keras noch die Größe der Batches an.

In [96]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, val_iterator, test_iterator = data.BucketIterator.splits((train, val, test), 
                                                                           batch_size = BATCH_SIZE,
                                                                           device = device)

### Z.2.3.3. Erstellung des Modells <a class="anchor" id="Z-2-3-3"/>

Anders als bei Keras stützt sich das Erstellen von Modellen auf Klassen, die wir selbst erstellen müssen. Dadurch wird die Menge an Code im Gegensatz zu Keras erhöht, dafür können wir auch viel besser unser Neuronales Netz auf unsere Bedürfnisse anpassen.

TODO: https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/1%20-%20Simple%20Sentiment%20Analysis.ipynb

Zum Vergleich die Erstellung eines Modells in Keras:<br>

```
model = models.Sequential()
model.add(layers.Dense(64, activation="relu", input_shape=(len(vocab),)))
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(len(np.unique(labels)), activation="softmax"))

model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])
```